In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

# 1. Generate some sample data
X = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]).reshape(-1, 1) # Features
y = np.array([2, 4, 5, 4, 5, 7, 8, 9, 10, 12]) # Target variable

# 2. Define the model
model = LinearRegression()

# 3. Train the model
model.fit(X, y)

# 4. Make a prediction
new_X = np.array([11]).reshape(-1, 1)
prediction = model.predict(new_X)

print(f"Trained model coefficient: {model.coef_[0]:.2f}")
print(f"Trained model intercept: {model.intercept_:.2f}")
print(f"Prediction for new data {new_X[0][0]}: {prediction[0]:.2f}")

In [ ]:
# pip install scikit-learn

## 1.text-classification

In [ ]:
from transformers import pipeline

In [ ]:
from transformers import pipeline

pipe = pipeline("text-classification")

In [ ]:
data = pipe("This restaurant is not awesome")
data[0]['label']


⭐ 3.6 Zero-Shot Classification

In [ ]:
pipe = pipeline("zero-shot-classification")

result = pipe(
    "This is a course about AI and machine learning.",
    candidate_labels=["education", "politics", "sports"]
)

print(result)


https://huggingface.co/docs/transformers/v4.57.0/en/main_classes/pipelines#transformers.Pipeline.model



## 2.Text Generations

In [ ]:
pipe = pipeline("text-generation", model="gpt2")

result = pipe(
    "Once upon a time in India, there was a data scientist",
    max_length=50,
    num_return_sequences=1
)

print(result)


In [ ]:
from transformers import pipeline

pipeline = pipeline(task="text-generation", model="google/gemma-2-2b")
pipeline("the secret to baking a really good cake is ")
[{'generated_text': 'the secret to baking a really good cake is 1. the right ingredients 2. the'}]

## 3. text summarizations

In [ ]:
from transformers import pipeline

pipeline = pipeline(task="summarization", model="google/pegasus-billsum")

In [ ]:
summarize_text = pipeline("Section was formerly set out as section 44 of this title. As originally enacted, this section contained two further provisions that 'nothing in this act shall be construed as in any wise affecting the grant of lands made to the State of California by virtue of the act entitled 'An act authorizing a grant to the State of California of the Yosemite Valley, and of the land' embracing the Mariposa Big-Tree Grove, approved June thirtieth, eighteen hundred and sixty-four; or as affecting any bona-fide entry of land made within the limits above described under any law of the United States prior to the approval of this act.' The first quoted provision was omitted from the Code because the land, granted to the state of California pursuant to the Act cite, was receded to the United States. Resolution June 11, 1906, No. 27, accepted the recession.")
[{'summary_text': 'Instructs the Secretary of the Interior to convey to the State of California all right, title, and interest of the United States in and to specified lands which are located within the Yosemite and Mariposa National Forests, California.'}]


summarize_text

In [ ]:
from transformers import pipeline

pipe = pipeline("sentiment-analysis")

result = pipe("I love using Hugging Face pipelines!")
print(result)

⭐ 3.5 Translation (English → Hindi)

In [ ]:

pipe = pipeline("translation_en_to_hi", model="Helsinki-NLP/opus-mt-en-hi")
print(pipe("I love learning transformers!"))

⭐ 3.7 Image Classification

In [ ]:
pipe = pipeline("image-classification", model="google/vit-base-patch16-224")

from PIL import Image
img = Image.open("/content/new_pic.jpg")

print(pipe(img))


In [ ]:
# ⭐ 3.8 Speech-to-Text (ASR)
pipe = pipeline("automatic-speech-recognition",
                model="openai/whisper-small")


# print(pipe("audio.wav"))
# https://huggingface.co/docs/transformers/en/model_doc/whisper

In [ ]:
pipe("https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac")

In [ ]:
# ⭐ 3.9 Embeddings / Feature Extraction
pipe = pipeline("feature-extraction",
                model="sentence-transformers/all-MiniLM-L6-v2")

emb = pipe("This is a feature extraction example.")
print(len(emb[0]))   # 384 dims


In [ ]:
 # hf_ CXeDvBPImWhKBaiJ
    # BFfuuKpqlQKxjIgIIA


## ⭐ 3.10 Text-to-Image

In [ ]:
# ⭐ 3.10 Text-to-Image

# (Pipeline does NOT support; must use Diffusers)

from diffusers import StableDiffusionPipeline
import torch

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to("cuda")

In [ ]:
image = pipe("a futuristic city in neon lights").images[0]
image.save("city.png")

In [ ]:
3

Alright, let’s build a **free image generation setup using Hugging Face + Transformers** 😎

I’ll show you two options:

1. **Completely free & easiest** → Use **Hugging Face Inference API** (no GPU needed, runs on HF servers)
2. **Local free (needs GPU/strong CPU)** → Download model and run via `transformers` / `diffusers`

Since you wrote *“using tranfrmer”*, I’ll keep it focused on Hugging Face ecosystem (Transformers-style pipelines).

---

## 1️⃣ Setup – Hugging Face Account & Token (One Time)

1. Go to Hugging Face → create a free account.
2. Go to **Settings → Access Tokens** → create a **Read token**.
3. Install required libraries:

```bash
pip install transformers torch pillow requests
```

If you want to use diffusion models (recommended for images):

```bash
pip install diffusers accelerate safetensors
```

---

## 2️⃣ Easiest Way: Use Hugging Face Inference API (Free Tier)

Here we **don’t download** the model; we just call it using HTTPS.
Example using a Stable Diffusion model (text → image):

```python
import requests

API_URL = "https://api-inference.huggingface.co/models/stabilityai/stable-diffusion-2-1"
HF_TOKEN = "YOUR_HF_API_TOKEN"   # <--- paste your token here

headers = {"Authorization": f"Bearer HF_TOKEN_REPLACE"}

def generate_image(prompt, output_path="generated.png"):
    payload = {"inputs": prompt}
    response = requests.post(API_URL, headers=headers, json=payload)

    if response.status_code != 200:
        print("Error:", response.status_code, response.text)
        return

    with open(output_path, "wb") as f:
        f.write(response.content)
    print(f"Saved image to {output_path}")


if __name__ == "__main__":
    prompt = "a futuristic city at sunset, ultra realistic, 4k"
    generate_image(prompt)
```

👉 **Important:** Replace

```python
headers = {"Authorization": f"Bearer HF_TOKEN_REPLACE"}
```

with

```python
headers = {"Authorization": f"Bearer " + HF_TOKEN}
```

This gives you a **free image generator** using a Hugging Face model, no GPU on your side.

---

## 3️⃣ Using `diffusers` + Transformers Locally (Open-Source, Free)

This uses a **Transformers-style pipeline** but via the `diffusers` library (Hugging Face official for diffusion models).

```python
from diffusers import StableDiffusionPipeline
import torch

# Download any free text-to-image model, example:
model_id = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  # use float32 if you don't have GPU
)

# If you have a GPU (recommended)
if torch.cuda.is_available():
    pipe = pipe.to("cuda")

prompt = "a cute robot teaching data science, digital art, vibrant colors"
image = pipe(prompt).images[0]

image.save("robot_ds.png")
print("Image saved as robot_ds.png")
```

If you **don’t** have GPU, change to:

```python
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float32
).to("cpu")
```

⚠️ CPU will be **very slow** but still free.

---

## 4️⃣ “Transformers” Style Pipeline (Non-diffusers models)

Some image models are available directly via `transformers` pipelines (e.g., text-to-image, image-to-image). Example:

```python
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image
import torch

# Example: Phi-3-Vision / Qwen-VL etc. can generate images or reason about them
# (Many models are more multimodal than pure generators)
```

But for **pure image generation**, **diffusers** is the normal + recommended way in HF ecosystem, and it’s built to work **with transformers under the hood**.

---

## 5️⃣ Wrap It as a Simple “Free HF Image Generator” Function

Here’s a neat utility that automatically uses GPU if available:

```python
from diffusers import StableDiffusionPipeline
import torch

class HFImageGenerator:
    def __init__(self, model_id="runwayml/stable-diffusion-v1-5"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        dtype = torch.float16 if self.device == "cuda" else torch.float32

        self.pipe = StableDiffusionPipeline.from_pretrained(
            model_id,
            torch_dtype=dtype,
        ).to(self.device)

    def generate(self, prompt: str, file_name: str = "output.png"):
        image = self.pipe(prompt).images[0]
        image.save(file_name)
        print(f"✅ Saved: {file_name}")

if __name__ == "__main__":
    gen = HFImageGenerator()
    gen.generate("a fantasy landscape with floating islands, matte painting", "fantasy.png")
```

This is basically a **free image generator model using Hugging Face** that you can plug into Flask, FastAPI, Streamlit, etc.

---

## 6️⃣ If You Tell Me Your Goal…

For example:

* “Use it in **Flask API**”
* “Use it in **Streamlit app**”
* “Use it in a **Jupyter Notebook** as a teaching demo”

…I can give you **full project code** (app.py + requirements.txt + sample prompts) tailored to that.

For now, you can copy-paste any of the above blocks and run them directly.
